# Submitting an experiment

In this notebook, we will use the Estimator object to submit an experiment to processing on a Azure-ML Compute Target.

An experiment is a process that can be tracked through its generated metrics.

In this particular case, we are using an external Python script to:

- get a dataset from Azure DataStorage
- create a machine learning pipeline for classification that includes:
    - grid search cross validation
    - logistic regression
    - decision trees
    - Random Forests
    - Gradient Boosting

## Imports


In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.train.estimator import Estimator

In [2]:
# load ws
ws = Workspace.from_config()

# Get the dataset
data = ws.datasets.get('poker_ds')

# Dataset head
data.take(10).to_pandas_dataframe()

C:\Users\BJ571WQ\Envs\azure\lib\site-packages\azureml\dataprep\api\dataflow.py:722: UserWarning: Your pandas and pyarrow versions are incompatible. Please install pyarrow>=0.12.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install azureml-dataprep[pandas].
  warnings.warn('Your pandas and pyarrow versions are incompatible. '


,Rank_1,Rank_2,Rank_3,Rank_4,Rank_5,class,Suit_1_2,Suit_1_3,Suit_1_4,Suit_2_2,...,Suit_2_4,Suit_3_2,Suit_3_3,Suit_3_4,Suit_4_2,Suit_4_3,Suit_4_4,Suit_5_2,Suit_5_3,Suit_5_4
0,11,13,10,12,1,9,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,12,11,13,10,1,9,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
2,10,11,1,13,12,9,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
3,1,13,12,11,10,9,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
4,2,4,5,3,6,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,9,12,10,11,13,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2,3,4,5,8,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
7,5,6,9,7,8,8,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
8,1,4,2,3,5,8,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
9,1,1,9,5,3,1,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0


In [3]:
# get compute target and start it
cpu_cluster = ComputeTarget(workspace=ws, name='pc3')
cpu_cluster.start()
cpu_cluster.wait_for_completion(show_output=True)


Starting......................
Running


## Logistic Regression

In [4]:
def run_experiment(chosen_model):
    # Create an estimator
    estimator = Estimator(
        source_directory='scripts',
        entry_script='experiment.py',
        compute_target='pc3',
        conda_packages=['scikit-learn'],
        inputs=[data.as_named_input('poker')],
        script_params = {'--chosen_model': chosen_model}
        )

    # Submit
    exp = Experiment(workspace=ws, name='PokerHand_Classification_Args')

    # Run the experiment based on the estimator
    run = exp.submit(config=estimator)
    RunDetails(run).show()
    run.wait_for_completion(show_output=True)
    return run

In [5]:
chosen_model = 'logistic'
run_experiment(chosen_model)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: PokerHand_Classification_Args_1591297079_ac88d7e6
Web View: https://ml.azure.com/experiments/PokerHand_Classification_Args/runs/PokerHand_Classification_Args_1591297079_ac88d7e6?wsid=/subscriptions/d8610c93-6c20-40ef-8ce5-281bf8b7f1d0/resourcegroups/erickfis-ml-rg/workspaces/poker-ws

Streaming azureml-logs/65_job_prep-tvmps_67202ecfa7825ff50d16661c26811eb9ab1462113ef082ee7c7de2e688fd5f9c_d.txt

Entering job preparation. Current time:2020-06-04T18:58:21.296235
Starting job preparation. Current time:2020-06-04T18:58:22.402045
Extracting the control code.
fetching and extracting the control code on master node.
Retrieving project from snapshot: 1cb3cf3d-dfbf-4cad-860b-bcdced4f9b1e
Starting the daemon thread to refresh tokens in background for process with pid = 40

Streaming azureml-logs/70_driver_log.txt

2020/06/04 18:58:26 Starting App Insight Logger for task:  runTaskLet
Entering context manager injector. Current time:2020-06-04T18:58:27.591721
Starting the daemon thread to re

/azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opt

/azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opt

Experiment,Id,Type,Status,Details Page,Docs Page
PokerHand_Classification_Args,PokerHand_Classification_Args_1591297079_ac88d7e6,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Decision trees

In [6]:
chosen_model = 'tree'
run_experiment(chosen_model)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: PokerHand_Classification_Args_1591297312_1c8ec24d
Web View: https://ml.azure.com/experiments/PokerHand_Classification_Args/runs/PokerHand_Classification_Args_1591297312_1c8ec24d?wsid=/subscriptions/d8610c93-6c20-40ef-8ce5-281bf8b7f1d0/resourcegroups/erickfis-ml-rg/workspaces/poker-ws

Streaming azureml-logs/65_job_prep-tvmps_67202ecfa7825ff50d16661c26811eb9ab1462113ef082ee7c7de2e688fd5f9c_d.txt

Entering job preparation. Current time:2020-06-04T19:02:13.715004
Starting job preparation. Current time:2020-06-04T19:02:14.443058
Extracting the control code.
fetching and extracting the control code on master node.
Retrieving project from snapshot: 3b5c8e9e-4d42-4ccb-8ad3-cc3d5dec0468
Starting the daemon thread to refresh tokens in background for process with pid = 42
Starting project file download.
Finished project file download.
downloadDataStore - Download from datastores if requested.
Entering context manager injector. Current time:2020-06-04T19:02:16.300109

Streaming azureml-log

Experiment,Id,Type,Status,Details Page,Docs Page
PokerHand_Classification_Args,PokerHand_Classification_Args_1591297312_1c8ec24d,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Randon Forests

In [7]:
chosen_model = 'rf'
run_experiment(chosen_model)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: PokerHand_Classification_Args_1591297404_63a590d9
Web View: https://ml.azure.com/experiments/PokerHand_Classification_Args/runs/PokerHand_Classification_Args_1591297404_63a590d9?wsid=/subscriptions/d8610c93-6c20-40ef-8ce5-281bf8b7f1d0/resourcegroups/erickfis-ml-rg/workspaces/poker-ws

Streaming azureml-logs/65_job_prep-tvmps_67202ecfa7825ff50d16661c26811eb9ab1462113ef082ee7c7de2e688fd5f9c_d.txt

Entering job preparation. Current time:2020-06-04T19:03:40.508149
Starting job preparation. Current time:2020-06-04T19:03:41.186214
Extracting the control code.
fetching and extracting the control code on master node.
Retrieving project from snapshot: d69c9937-ab48-4b97-ae09-ff4a39da97b9
Starting the daemon thread to refresh tokens in background for process with pid = 43
Starting project file download.

Streaming azureml-logs/70_driver_log.txt

2020/06/04 19:03:44 Starting App Insight Logger for task:  runTaskLet
Entering context manager injector. Current time:2020-06-04T19:03:46.428807


Experiment,Id,Type,Status,Details Page,Docs Page
PokerHand_Classification_Args,PokerHand_Classification_Args_1591297404_63a590d9,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Gradient Boosting

In [8]:
chosen_model = 'gb'
run_experiment(chosen_model)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: PokerHand_Classification_Args_1591300637_d632f2ff
Web View: https://ml.azure.com/experiments/PokerHand_Classification_Args/runs/PokerHand_Classification_Args_1591300637_d632f2ff?wsid=/subscriptions/d8610c93-6c20-40ef-8ce5-281bf8b7f1d0/resourcegroups/erickfis-ml-rg/workspaces/poker-ws

Streaming azureml-logs/65_job_prep-tvmps_67202ecfa7825ff50d16661c26811eb9ab1462113ef082ee7c7de2e688fd5f9c_d.txt

Entering job preparation. Current time:2020-06-04T19:57:32.670091
Starting job preparation. Current time:2020-06-04T19:57:33.354078
Extracting the control code.
fetching and extracting the control code on master node.
Retrieving project from snapshot: 1948e57c-6cd6-41d1-ae33-4f1b529fab2c
Starting the daemon thread to refresh tokens in background for process with pid = 40
Starting project file download.
Finished project file download.
downloadDataStore - Download from datastores if requested.
Entering context manager injector. Current time:2020-06-04T19:57:35.194676

Streaming azureml-log

WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F6E01F5390>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond')': /history/v1.0/subscriptions/d8610c93-6c20-40ef-8ce5-281bf8b7f1d0/resourceGroups/erickfis-ml-rg/providers/Microsoft.MachineLearningServices/workspaces/poker-ws/experiments/PokerHand_Classification_Args/runs/PokerHand_Classification_Args_1591300637_d632f2ff/details
WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))': /azureml/ExperimentRun/dcid.PokerHand_Classification_Args_1591300637_d632f2ff/azureml-logs/70_driver_log.txt?


Streaming azureml-logs/75_job_post-tvmps_67202ecfa7825ff50d16661c26811eb9ab1462113ef082ee7c7de2e688fd5f9c_d.txt

Entering job release. Current time:2020-06-04T20:13:17.926214
Starting job release. Current time:2020-06-04T20:13:19.156188
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 387
Entering context manager injector. Current time:2020-06-04T20:13:19.177856
Job release is complete. Current time:2020-06-04T20:13:20.446830

Execution Summary
RunId: PokerHand_Classification_Args_1591300637_d632f2ff
Web View: https://ml.azure.com/experiments/PokerHand_Classification_Args/runs/PokerHand_Classification_Args_1591300637_d632f2ff?wsid=/subscriptions/d8610c93-6c20-40ef-8ce5-281bf8b7f1d0/resourcegroups/erickfis-ml-rg/workspaces/poker-ws



Experiment,Id,Type,Status,Details Page,Docs Page
PokerHand_Classification_Args,PokerHand_Classification_Args_1591300637_d632f2ff,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## View Metrics for last run

In [44]:
exp = ws.experiments['PokerHand_Classification_Args']
all_runs = exp.get_runs()
all_runs = list(all_runs)
last_run = all_runs[0].get_metrics()
last_run

{'Gradient Boosting': "{'model name': 'Gradient Boosting', 'train_score': 0.6578883637880406, 'test_score': 0.6674663467946155, 'parameters': {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 500, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': 95276, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}}",
 'accuracy': 0.6674663467946155}

## Stopping the compute target

In [45]:
for pc_name in ws.compute_targets:
    pc = ws.compute_targets[pc_name]
    pc.stop()

